# sample whisper promptping

In [ ]:
!pip install git+https://github.com/Hasan-Naseer/whisperX.git@release/latest-faster-whisper-version
!pip install ctranslate2==4.4.0

In [ ]:
!apt-get install libcudnn8 libcudnn8-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  libcudnn8 libcudnn8-dev
0 upgraded, 2 newly installed, 0 to remove and 20 not upgraded.
Need to get 885 MB of archives.
After this operation, 2,380 MB of additional disk space will be used.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  libcudnn8 8.9.7.29-1+cuda12.2 [444 MB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  libcudnn8-dev 8.9.7.29-1+cuda12.2 [440 MB]
Fetched 885 MB in 9s (96.3 MB/s)
Selecting previously unselected package libcudnn8.
(Reading database ... 124926 files and directories currently installed.)
Preparing to unpack .../libcudnn8_8.9.7.29-1+cuda12.2_amd64.deb ...
Unpacking libcudnn8 (8.9.7.29-1+cuda12.2) ...
Selecting previously unselected package libcudnn8-dev.
Preparing to unpack .../libcudnn8-dev_8.9.7.29-1+cuda12.2_amd64.deb ...
Unpacking libcudnn8-d

In [ ]:
from huggingface_hub.utils import _runtime
_runtime._is_google_colab = False

In [ ]:
import whisperx
import gc

device = "cuda"
audio_file = "/content/drive/MyDrive/REDC/Test/fn000049_1_003.wav"
batch_size = 8 # reduce if low on GPU mem
compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)

# 1. Transcribe with original whisper (batched)
#model = whisperx.load_model("large-v2", device, compute_type=compute_type,initial_prompt='dat da dat')

# save model to local path (optional)
# model_dir = "/path/"
# model = whisperx.load_model("large-v2", device, compute_type=compute_type, download_root=model_dir)


/usr/local/lib/python3.11/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
/usr/local/lib/python3.11/dist-packages/pyannote/audio/pipelines/speaker_verification.py:43: UserWarning: torchaudio._backend.get_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  backend = torchaudio.get_audio_backend()
/usr/local/lib/python3.11/dist-packages/pyannote/audio/pipelines/speaker_verification.py:45: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  from speechbrain.pretrained import (
/usr/local/lib/python3.11/dist-packages/pyannote/audio/pipelines/speaker_

In [ ]:
import whisperx
model = whisperx.load_model("large-v3", device, language='nl',compute_type=compute_type, asr_options={"hotwords":None,"initial_prompt": "eerst ee rst eer st ehij geeft ze allemaal een naam eerste was hhij geeft ze allema ehij geeft ze allemaal een naamt et p lagened plad gen flagen plagen dat vond maar tje maar maarte maart t maartje wel leuk maar l eu k loek luk maar naar maar toen het pesten pe te sten werd is ze mevrouw fre f r e d d e r fre dder fledder vledder gaan hel pen h e l p e n gaan helpen hand gebaren handgebaren"})
audio_file = "/content/drive/MyDrive/REDC/Trim_Jasmin_Testing_Audio/fn000503/1.wav"
audio = whisperx.load_audio(audio_file)
result = model.transcribe(audio=audio,batch_size=batch_size)
print(result["segments"]) # before alignment

#print(result) # before alignment

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../usr/local/lib/python3.11/dist-packages/whisperx/assets/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.5.1+cu124. Bad things might happen unless you revert torch to 1.x.
[{'text': ' hij geeft ze allemaal hij hij geeft ze allemaal een naam', 'start': 0.009, 'end': 4.053}]


In [ ]:
result = model.transcribe(audio=audio,batch_size=batch_size)
print(result["segments"]) # before alignment

# normalization

In [ ]:
import pandas as pd
import os
import numpy as np
import json

Reformat_path = "/content/drive/MyDrive/REDC/Jasmin_Child_Reformat_SanneAnno/"
children_data_path = "/content/drive/MyDrive/REDC/JASMIN_textgrid/"
child_speaker_file = os.path.join(children_data_path+"sprekers_informatie.xlsx")
child_data = pd.read_excel(child_speaker_file, index_col=0, usecols = np.arange(0,11))

#long word can lead to index error in word_transcripiton



DUTCH_NORMALIZATION = {
    "okee": "ok", "oke": "ok", "okey": "ok", "OKEE": "ok", "OKE": "ok",
    "OKEY": "ok", "MIJN": "mijn", "mn": "mijn", "m'n": "mijn", "M'N": "mijn", "mjn": "mijn",
    "'t": "het", "'T": "het", "'n": "een", "Een": "een", "EEN": "een", "z'n": "zijn", "zn": "z'n",
    "'m":"hem",
    "d'r": "daar", "dr": "daar",
    "'s": "s", "zo'n": "zoon",
    "'k": "ik", "'K": "ik",
    "uh": "", "ggg":"","mmm":"",
    "s nachts": "s'nachts",
}

def normalize_Dutch_string(input_string):
    normalized_string = input_string
    for key, value in DUTCH_NORMALIZATION.items():
        normalized_string = normalized_string.replace(key, value)
    return normalized_string

def normalize_long_words(input_string):
    normalized_string = input_string
    for key, value in Long_word.items():
        normalized_string = normalized_string.replace(key, value)
    return normalized_string

def read_phone_file(filename):
    phone = []
    with open(filename, 'r') as file:
        for line in file:
            # Split each line into key and value
            phone.append(line.strip())

    return " ".join(phone)

def reverse(input_string):
    txt = input_string.split(" ") [::-1]
    return " ".join(txt)

def text_norm_all(s):
    # Convert to lowercase
    s = s.lower()

    # Replace tag_a words
    #s = s.replace("<tag_a>", "<unk>")  # Assuming <tag_a> is the tag to be replaced
    s = s.replace("*n", "")  # Remove *a
    s = s.replace("*a", "")  # Remove *a
    s = s.replace("*u", "")  # Remove *a
    s = s.replace("*s", "")  # Remove *a
    s = s.replace("*v", "")  # Remove *

    # Remove punctuation
    punctuation = ".?!,-"
    for p in punctuation:
        s = s.replace(p, "")

    s = s.replace("’", "'")




    s =  normalize_Dutch_string(s)


    return s